# Creating the database

In [1]:
import glob
import os
import re
import pandas as pd

In [10]:
TournamentData = pd.read_csv('Alle_Tuniere_final.csv', sep=';')
TournamentData['No'] = TournamentData['No'].astype('object')

### read csv-files with tournament-statistic

In [ ]:
# CSV-Dateien im aktuellen Verzeichnis finden
csv_files = glob.glob("tournament_statistics_*.csv")

df_list_30 = []
df_list_mixed = []

for file in csv_files:
    df = pd.read_csv(file, sep='\t')
    
    # Turniernummer aus Dateinamen extrahieren
    match = re.search(r'tournament_statistics_(\d+)\.csv', os.path.basename(file))
    if match:
        df["TournamentNo"] = match.group(1)
    else:
        df["TournamentNo"] = None

    # Unterscheidung nach Typ
    if 'ItemType' in df.columns:
        types = df['ItemType'].astype(str).unique()
        #files without statistic data
        if set(types) == {'30'}:
            df_list_30.append(df)
        #files with statistic data
        elif '11' in types:
            df_list_mixed.append(df)
        else:
            print(f"Unerwartete Typen in {file}: {types}")
    else:
        print(f"'Type' column missing in {file}, skipping...")

# Zu einem DataFrame zusammenfügen
combined_30 = pd.concat(df_list_30, ignore_index=True)
combined_mixed = pd.concat(df_list_mixed, ignore_index=True)

combined_mixed['NoMatch'] = combined_mixed['NoMatch'].astype(str)

In [ ]:
BeachMatches = pd.read_csv('BeachMatchList_full.csv', sep=';')
BeachMatches['@No'] = BeachMatches['@No'].astype(str)
Combined_Stat = combined_mixed.merge(BeachMatches, left_on = 'NoMatch', right_on='@No', how= 'left')

In [ ]:
TournamentData["No"] = TournamentData["No"].astype(str)
Combined_Stat["TournamentNo"] = Combined_Stat["TournamentNo"].astype(str)

In [35]:
Dat_Merge = TournamentData.merge(Combined_Stat,  left_on='No',right_on='TournamentNo', how='left',
    suffixes=('_tourn', '_stat'))

### Preparing data to add coordinates

Cleaning of variables: CountryName, Name, DefaultCity and Title (of the Tournament)

In [ ]:
def process_default_city(dat_merge):
 
   # 1. Separate rows with and without 'DefaultCity'
    city_full = dat_merge[dat_merge['DefaultCity'].notna()].copy()
    city_na = dat_merge[dat_merge['DefaultCity'].isna()].copy()
    
    # 2. Fill missing 'DefaultCity' values using 'Name_tourn'
    city_na['DefaultCity'] = city_na['Name_tourn']
    
    # Copy for transformation
    df = city_na.copy()
    df["Ort"] = pd.Series(dtype="object")
    
   # 3a. If 'DefaultCity' consists of a single word (without spaces), set it directly
    mask_single = df["DefaultCity"].str.strip().str.count(r'\s') == 0
    df.loc[mask_single, "Ort"] = df.loc[mask_single, "DefaultCity"].str.strip()
    
   # 3b. If 'DefaultCity' contains " - ", extract the part after the last " - "
    mask_dash = df["Ort"].isna() & df["DefaultCity"].str.contains(" - ")
    df.loc[mask_dash, "Ort"] = df.loc[mask_dash, "DefaultCity"] \
                                .str.extract(r" - ([^-\n\r]+)$")[0].str.strip()
    
   # 3c. Fill remaining missing values in 'Ort'
    df["Ort"] = df["Ort"].fillna("Unbekannt")
    
   # 4. Generate a category code for 'DefaultCity'
    df['ID_DefaultCity'] = df['DefaultCity'].astype('category').cat.codes

   # Load mapping DataFrame
    Dat_ort = pd.read_csv('Ort_Noch_anpassen.csv', sep=';')

   # 5. Merge with the mapping DataFrame (assumption: Column 0 = ID_DefaultCity, Column 2 = Ort_Neu)
    df_merged = df.merge(Dat_ort.iloc[:, [0, 2]], on='ID_DefaultCity', how='left')
    
   # 6. Overwrite 'DefaultCity' with the mapped value 'Ort_Neu'
    df_merged['DefaultCity'] = df_merged['Ort_Neu']
    
   # 7. Remove helper columns
    df_final_subset = df_merged.drop(columns=['Ort', 'ID_DefaultCity', 'Ort_Neu'])
    
   # 8. Merge the processed rows back with the original ones where 'DefaultCity' was already set
    final_df = pd.concat([city_full, df_final_subset], ignore_index=True)

    DatFinal1 = final_df[final_df['@LocalTime'].notna()]
    
    return DatFinal1


In [40]:
database = process_default_city(Dat_Merge)